***
<center><font color="dodgerblue"><font size="7"><b><i>Analyse de Données Développeur<br />(C5-160512-INFO)</i></b></font></font></center>

***

<center><font color="darkorange"><font size="8"><b><i>4) Apprentissage Supervisé (TP #1) <br/><font size="6">&copy C. Frélicot, Automne 2023</font></i></b></font></font></center>

***

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from numpy.matlib import repmat
import warnings
warnings.filterwarnings('ignore')

***
# 0)  Tableau de données (du TD)
***
## Lecture de fichier formatté à l'aide de *Pandas*.

In [3]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
learndf = pd.read_excel("TD4-2023.xls",sheet_name="LearningSet",index_col=0)
X = learndf.values[:,:2]
Ytrue = learndf.values[:,2]
learn_label = learndf.index.values
learndf

,$X_1$,$X_2$,$Y$
(1),0,2,Class 1
(2),2,0,Class 1
(3),2,6,Class 1
(4),4,4,Class 1
(5),2,4,Class 2
(6),6,7,Class 2
(7),10,4,Class 2
(8),4,0,Class 3
(9),4,2,Class 3
(10),6,0,Class 3


In [4]:
testdf = pd.read_excel("TD4-2023.xls",sheet_name="PredictionSet",index_col=0)
xyz = testdf.values[:,:2]
test_label = testdf.index.values
testdf

,$X_1$,$X_2$
$x$,3,2
$y$,10,2
$z$,5,3


***
# 1) Nearest-Prototype Rule
***

## $(1) \ L : x\mapsto u(x)=[u_j(x)]_{j=1,c}$ tel que $u_j(x) = d(x,\overline{x}_j)$
## $(2) \ A : x\mapsto y(x) = argmin_j \ u_j$
***

## 1-1) Pensez-vous avoir toutes les fonctions nécessaires au codage de cette règle de classification supervisée ? Si oui, copiez-les ci-dessous. 

In [17]:
def prototyping(data, indicator):
    """
    Calcule les barycentres des clusters d'un tableau de données.

    Paramètres:
        data (numpy.ndarray): Tableau des données (n individus x p variables).
        indicator (array-like): Indicatrice de partition (taille n).

    Retourne:
        dict: Dictionnaire des barycentres, avec les indices des clusters comme clés.
    """
    indicator = np.array(indicator)  # Assure que l'indicatrice est un tableau numpy
    unique_clusters = np.unique(indicator)
    barycenters = {}

    for cluster in unique_clusters:
        cluster_points = data[indicator == cluster]  # Filtrage correct
        barycenters[cluster] = np.mean(cluster_points, axis=0)

    return barycenters


def dist(x,y,dname='euclidean'):
    if dname == 'manhattan':
        d = np.sum(np.abs(x-y))
    elif dname == 'euclidean':
        d = np.sqrt(np.sum((x-y)**2))
    elif dname == 'chebyshev':
        d = np.max(np.abs(x-y))
    elif dname == 'cosine':
        d = 1 - np.dot(x,y)/(np.linalg.norm(x)*np.linalg.norm(y))
    elif dname == 'correlation':
        d = 1 - np.dot(x-np.mean(x),y-np.mean(y))/(np.linalg.norm(x-np.mean(x))*np.linalg.norm(y-np.mean(y)))
    
    return d

## 1-2) Ecrivez une fonction *NPClassifier* permettant de prédire, pour tous les points d'un tableau *X_pred*, leur prédiction *y_pred* avec la règle du Plus Proche Prototype sur un ensemble d'apprentissage (*X_learn*, *y_learn*) au sens d'une distance *dname* (euclidienne par défaut).

## Vous la testerez sur les données de TD de sorte de trouver les résultats, y compris les (HW).

In [13]:
X_learn = X
y_learn = Ytrue
X_pred = xyz

In [18]:
def NPClassifier(X_learn, y_learn, X_pred, dname='euclidean'):
    """
    Prédit les étiquettes pour X_pred en utilisant la règle du Plus Proche Prototype.

    Paramètres:
        X_learn (numpy.ndarray): Données d'apprentissage (n x p).
        y_learn (array-like): Étiquettes d'apprentissage (taille n).
        X_pred (numpy.ndarray): Données à prédire (m x p).
        dname (str): Nom de la distance ('euclidean' par défaut).

    Retourne:
        list: Étiquettes prédites pour X_pred.
    """
    # Calcul des prototypes (barycentres) pour chaque classe
    prototypes = prototyping(X_learn, y_learn)

    # Prédiction pour chaque point de X_pred
    y_pred = []
    for x in X_pred:
        distances = {label: dist(x, prototype) for label, prototype in prototypes.items()}
        closest_label = min(distances, key=distances.get)
        y_pred.append(closest_label)

    return y_pred

In [19]:
NPClassifier(xyz,X,Ytrue)

IndexError: boolean index did not match indexed array along dimension 0; dimension is 3 but corresponding boolean dimension is 11

In [ ]:
NPClassifier(xyz,X,Ytrue,'manhattan')

In [ ]:
NPClassifier(xyz,X,Ytrue,'chebychev')

In [ ]:
NPClassifier(xyz,X,Ytrue,'cosine')

## 1-3) Reclassez les données d'apprentissage par la règle *NP*. 

## Définissez la matrice de confusion de sorte de voir si tous les points sont-ils bien reclassés.

***
# 2) K Nearest-Neighbors Rule
***

## $(0) \ D(x) = [d_{(1)}, d_{(2)}, ... d_{(K)}]$ 
## $\quad \ Y_{NN} = [Y_{(1)}, Y_{(2)}, ... Y_{(K)}]$
## $(1) \ L : x\mapsto u(x)=[u_j(x)]_{j=1,c}$ tel que $u_j(x) = k_j / K$ où 
## $\quad \ k_j(x) = card(Y_{NN}==j)$
## $(2) \ A : x\mapsto y(x) = argmax_j \ u_j$
***

## 2.1) Procédons tout d'abord pas à pas.
## Pour le calcul des distances entre les points d'apprentissage et ceux à prédire, vous utiliserez la fonction ci-dessous

In [20]:
def twosets_dist(X1,X2,dname='euclidean'):
    dim1 = len(np.shape(X1))
    dim2 = len(np.shape(X2))
    # swap arrays if the second one is less-dimensional
    if dim2 < dim1:
        dim = dim2 ; dim2 = dim1 ; dim1 = dim
        X = X2 ; X2 = X1 ; X1 = X      
    # if first array is one-dimensional
    if dim1<2:
        if dim2<2:
        # if both are
            return dist(X1,X2,dname)
        else:
            distances = np.zeros(X2.shape[0])
            for l in range(X2.shape[0]):
                distances[l] = dist(X1,X2[l],dname)
            return distances
    # else
    distances = np.zeros((X1.shape[0],X2.shape[0]))
    for k in range(X1.shape[0]):
        for l in range(X2.shape[0]):
            distances[k,l] = dist(X1[k],X2[l],dname)
    return distances

In [21]:
twosets_dist(X_pred,X)**2

array([[  9.,   5.,  17.,   5.,   5.,  34.,  53.,   5.,   1.,  13.,   9.],
       [100.,  68.,  80.,  40.,  68.,  41.,   4.,  40.,  36.,  20.,  16.],
       [ 26.,  18.,  18.,   2.,  10.,  17.,  26.,  10.,   2.,  10.,   2.]])

## Etape (0) : boucle sur les points à prédire où l'on sélectionne leurs *K-NN* et les valeurs d'indicatrice $Y_{NN}$

In [22]:
X_learn = X
y_learn = Ytrue
X_pred = xyz

In [ ]:
test_nb = X_pred.shape[0]
for t in range(test_nb):

    
    
    

## Etape (L) : ajoutez dans la boucle le calcul des votes et étiquettes $u$ ("probabilités" dans Orange 3)

In [ ]:
test_nb = X_pred.shape[0]
class_id = np.unique(y_learn)
class_nb = len(class_id)
for t in range(test_nb):

    
    
    
    
    
    

## 2-2) Reprenez le code précédent pour écrire une fonction *NNClassifier* permettant de prédire, pour tous les points d'un tableau *X_pred*, leur prédiction *y_pred* avec la règle des K (=1 par défaut) Plus Proches Voisins sur un ensemble d'apprentissage (*X_learn*, *y_learn*) au sens d'une distance *dname* (euclidienne par défaut).

In [ ]:
def NNClassifier(X_pred, X_learn, y_learn, K=1, dname='euclidean'):

    
    
    
    
    
    
    
    
    
    
    return y_pred

## Vous la testerez sur les données de TD de sorte de trouver les résultats, y compris les (HW).

## 2-3) Reclassez les données d'apprentissage par la règle du 1-NN. Tous les points sont-ils bien reclassés ? 

## Pourquoi un tel résultat ?

***
# 3) Evaluation
***

## 3-1) Ecrivez une fonction *classif_scores* qui retourne le taux de bon classement (Classification Accuracy) à partir d'une matrice de confusion. Vous la testerez en reclassant les données d'apprentissage selon la règle *NP*.

In [ ]:
def classif_scores(conf_matrix):

    
    
    return CA

## 3-2) Modifiez votre fonction de sorte qu'elle retourne aussi les scores globaux de rappel (*Recall*) et de précision (*Precision*) par classe

In [ ]:
def classif_scores(conf_matrix):

    
    
    
    
    
    return CA, recalls, precisions